In [1]:
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

Загружаем данные по полям

In [2]:
fields = pd.read_csv('Dataset/fields_merged.csv')
fields.head()

,field,region,land_type,fertility_score,season,crop,crop_predecessor,crop_yearold,area,hybrid,...,seed_origin,technology,yield_plan,p2o5,k2o,humus,ph,absolut,area_y,yield_fact
0,22.47.01.001.01,Алтай,пашня,NaN,2023,Оз.пшеница мягкая,Пар; Яр.пшеница мягкая,NaN,306.22,Скипетр,...,Покупные,минимальная,3.8,221.05,206.3,4.05,5.25,755.01510,306.22,2.465597
1,22.47.01.001.02,Алтай,пашня,NaN,2023,Пар,Пар; Яр.пшеница мягкая,NaN,63.16,NaN,...,NaN,минимальная,NaN,221.05,206.3,4.05,5.25,NaN,NaN,NaN
2,22.47.01.002.00,Алтай,пашня,4.0,2023,Пар,Яр.пшеница мягкая,NaN,485.79,NaN,...,NaN,NaN,NaN,199.80,186.0,3.50,5.20,NaN,NaN,NaN
3,22.47.01.003.00,Алтай,пашня,4.0,2023,Оз.пшеница мягкая,Пар,NaN,452.25,Скипетр,...,Покупные,минимальная,3.8,186.50,174.8,5.90,5.40,1177.06750,452.25,2.602692
4,22.47.01.004.00,Алтай,пашня,3.0,2023,Оз.пшеница мягкая,Пар,NaN,269.64,Скипетр,...,Покупные,минимальная,3.8,166.60,147.9,5.60,5.40,764.96084,269.64,2.836971


In [3]:
def interquantile_remove(df, columns):
    for column in columns:
        if "is_anomaly_" + column in df.columns:
            df_filtered = df[~df["is_anomaly_" + column]]
        else:
            df_filtered = df

        Q1 = df_filtered[column].quantile(0.25)
        Q3 = df_filtered[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df["is_anomaly_" + column] = ~((df[column] >= lower_bound) & (df[column] <= upper_bound))

    return df

Отмечаем пустые поля за аномалии

In [4]:
for miner in ['p2o5', 'k2o', 'ph', 'humus', 'yield_plan', 'yield_fact']:
    fields[f"is_anomaly_{miner}"] = fields[miner].isna()

In [5]:
print("Удалено NaN по хим элементам:", len(fields[fields['p2o5'].isna() | fields['k2o'].isna() | fields['ph'].isna() | fields['humus'].isna()]))
print("Удалено NaN по план урожайности:", len(fields[fields['yield_plan'].isna()]))
print("Удалено NaN по факт урожайности:", len(fields[fields['yield_fact'].isna()]))

Удалено NaN по хим элементам: 5797
Удалено NaN по план урожайности: 5205
Удалено NaN по факт урожайности: 11298


Находим выбросы методом интерквартильного размаха

In [6]:
fields = interquantile_remove(fields, ['area', 'p2o5', 'k2o', 'humus', 'ph', 'yield_plan', 'yield_fact'])

In [7]:
fields['count_anomaly'] = fields['is_anomaly_p2o5'].astype(int) + fields['is_anomaly_k2o'].astype(int) + fields['is_anomaly_ph'].astype(int) + fields['is_anomaly_humus'].astype(int) + fields['is_anomaly_yield_plan'].astype(int)

In [8]:
cleaned = fields[fields['is_anomaly_p2o5'] | fields['is_anomaly_k2o'] | fields['is_anomaly_ph'] | fields['is_anomaly_humus']]
cleaned_plan = fields[fields['is_anomaly_yield_plan']]
cleaned_fact = fields[fields['is_anomaly_yield_fact']]
cleaned_all = fields[(fields['count_anomaly'] > 0) | fields['is_anomaly_yield_fact']]

print('Выбросы по хим элементам:', len(cleaned))
print('Выбросы по план урожайности:', len(cleaned_plan))
print('Выбросы по факт урожайности:', len(cleaned_fact))
print("Всего выбросов:", len(cleaned_all))
print()
print("Было:", len(fields))
print("Осталось:", len(fields) - len(cleaned_all))

Выбросы по хим элементам: 6330
Выбросы по план урожайности: 5386
Выбросы по факт урожайности: 11394
Всего выбросов: 12446

Было: 16060
Осталось: 3614


Сохраняем датасет

In [9]:
fields.to_csv("Dataset/fields_anomaly.csv", index=False)